In [1]:
#Imports
import os
os.environ["OMP_NUM_THREADS"] = '1'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates

In [2]:
url='https://docs.google.com/spreadsheets/d/e/2PACX-1vQN3Ksa9szQuO4G1-msXWAp17KtVHQCBnuEieu_auu1wSiBf3-krHusIx5VBMkihxj-KZLBosDIGEyR/pub?output=csv'

In [ ]:
df=pd.read_csv(url)

In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
df['SizeRank'].unique()

In [ ]:
# larger city=lower rank.
largest_city_ranks = [0, 1, 2, 3]
large_city = df[df['SizeRank'].isin(largest_city_ranks)]

In [ ]:
# Id vars
id_variables=['RegionID','SizeRank','RegionName','RegionType',
              'StateName','State','Metro','CountyName']
#Melt
melted=pd.melt(large_city,id_vars=id_variables, var_name='Date',
               value_name='HomeValue')
#convert date to datetime.
melted['Date']=pd.to_datetime(melted['Date'])
#Set index
melted.set_index('Date',inplace=True)
#resample
monthly=melted.resample('M').mean(numeric_only=True)

In [ ]:
monthly['SizeRank'] = monthly['SizeRank'].astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Iterate through each size rank
for size_rank in largest_city_ranks:
    city_data = monthly[monthly['SizeRank'] == size_rank]
    ax.plot(city_data.index, city_data['HomeValue'], label=f'City {size_rank}')

# Set x-axis and y-axis labels
ax.set_xlabel('Date')
ax.set_ylabel('Average Home Value')
ax.set_title('Average Home Values for the Four Largest Cities')

# Add legend
ax.legend()

# Format y-axis tick labels as K (thousands)
def format_ticks(value, tick_number):
    return f'{value/1000:.0f}K'
ax.yaxis.set_major_formatter(FuncFormatter(format_ticks))

# Format x-axis date labels
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.YearLocator())

# Rotate x-axis date labels for better readability
fig.autofmt_xdate()

# Show the plot
plt.show()

In [ ]:
monthly.head()

In [ ]:
#Filter for 2008
dec_2008 = monthly.loc['2008-12-31']
#sort values
sorted_cities=large_city.sort_values(by='2008-12-31',ascending=False)
#iloc for highest and lowest.
highest=sorted_cities.iloc[0]['RegionName']
lowest=sorted_cities.iloc[-1]['RegionName']

# Which City had the highest Typical Home Value at the end of 2008? Which had the least?

In [ ]:
print("City with the highest typical home value at the end of 2008:",highest)
print("City with the least typical home value at the end of 2008:",lowest)

# How much did the home values change from November 2008 to December 2008 (in dollars)?

In [ ]:
change2008 = monthly['HomeValue'].diff(periods=1).loc['2008-12-31']
print("Change in home values from November 2008 to December 2008:",change2008)